Creating basic API call from Alpaca

In [1]:
# importing requisite libraries
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
import json

In [2]:
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


True

In [3]:
ticker_symbol = ["JPM"]

In [7]:
def fetch_ohlcv(ticker, start_date= "2020-01-01", end_date = "2021-01-01"):
    """Function to automatically pull closing price data from Alpaca"""
    """  example syntax: get_dat_data("TSLA")  """
    
    # Insert Personal Secret Keys (check .ENV to match syntax if error occurs)
    alpaca_api_key = os.getenv("APCA_API_KEY_ID")
    alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
    # Create the Alpaca API object
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version= "v2")
    
    # Backdating the Data to the first day of 2020
    back_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()

    # Choosing the ticker we want (need to figure out how to make this the input)
    #ticker = ["TSLA"]
    
    # Using our assumption of 15min intervals for trading
    timeframe = "15Min"
    
    # Creating a dataframe with every characteristic of ticker Alpaca allows
    df_ticker = alpaca.get_barset(
        ticker,
        timeframe,
        start=back_date,
        end = end_date
        ).df
    
    return df_ticker[ticker]

In [6]:
fetch_ohlcv("AAPL")

,open,high,low,close,volume
2020-01-02 09:30:00-05:00,296.280,297.950,295.210,297.130,67089
2020-01-02 09:45:00-05:00,297.190,298.150,296.710,298.000,29727
2020-01-02 10:00:00-05:00,297.975,298.400,297.370,297.370,40507
2020-01-02 10:15:00-05:00,297.355,297.580,296.620,297.460,32067
2020-01-02 10:30:00-05:00,297.540,297.750,297.130,297.620,31451
...,...,...,...,...,...
2020-12-31 15:00:00-05:00,133.000,133.180,132.870,132.955,90980
2020-12-31 15:15:00-05:00,133.000,133.045,132.575,132.785,96906
2020-12-31 15:30:00-05:00,132.805,132.850,132.660,132.720,82904
2020-12-31 15:45:00-05:00,132.710,133.300,132.520,132.580,462518


In [ ]:
def practice_ohlcv(ticker):
    """Function to automatically pull closing price data from Alpaca"""
    """  example syntax: get_dat_data(["TSLA"])  """
    
    # Insert Personal Secret Keys (check .ENV to match syntax if error occurs)
    alpaca_api_key = os.getenv("APCA_API_KEY_ID")
    alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
    # Create the Alpaca API object
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version= "v2")
    
    # Backdating the Data to the first day of 2020
    back_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
    
    # Using our assumption of 15min intervals for trading
    timeframe = "15Min"
    
    # Creating a dataframe with every characteristic of ticker Alpaca allows
    df_ticker = alpaca.get_barset(
        ticker,
        timeframe,
        start=back_date
        ).df
    
    ohlc_df = pd.DataFrame(df_ticker)
    # ohlc_df = ohlc_df.rename(columns={"open":"Open", "high":"High", "low":"Low", "close":"Close", "volume":"Volume"})
    return ohlc_df

In [ ]:
# Bollinger Band function to capture signal
def bollinger_band_signal_generator(dataframe_name, closing_price_column_name = 'close', bollinger_band_window = 20, num_standard_deviation = 2):
    """Creates Bollinger Band indicator with signal for long position based on closing price
    Args:
        dataframe_name (dict): Single security dataframe containing at least closing prices
        closing_price_column_name (str): Name of column in dataframe containing closing prices
        bollinger_band_window (int): Desired timeframe window used for rolling calculations
        num_standard_deviation (int): Desired number of standard deviations to calculate
    Returns:
        A dataframe of:
            original data passed to function,
            bollinger_band_middle (flt): Column of values for middle band,
            bollinger_band_std (flt): Column of values to calculate standard deviation,
            bollinger_band_upper (flt): Column of values for upper band,
            bollinger_band_lower (flt): Column of values for lower band,
            bollinger_band_long (flt): Column of generated signals (1.0 = True, 0.0 = False)
    """
    # Calculate mean and standard deviation
    dataframe_name['bollinger_band_middle'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).mean()
    dataframe_name['bollinger_band_std'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).std()

    # Calculate upper bollinger band and lower bollinger band
    dataframe_name['bollinger_band_upper'] = dataframe_name['bollinger_band_middle'] + (dataframe_name['bollinger_band_std'] * num_standard_deviation)
    dataframe_name['bollinger_band_lower'] = dataframe_name['bollinger_band_middle'] - (dataframe_name['bollinger_band_std'] * num_standard_deviation)

    # Create signal column
    dataframe_name['bollinger_band_long'] = np.where(dataframe_name[closing_price_column_name] > dataframe_name['bollinger_band_upper'], 1.0, 0.0)
    
    # Drop NaN values
    dataframe_name.dropna(inplace=True)

    # Return dataframe with features and target
    return dataframe_name


In [ ]:
bollinger_band_signal_generator(fetch_close(ticker_symbol), "close").tail()

In [ ]:
fetch_ohlc = fetch_ohlc(ticker_symbol)
fetch_ohlc["JPM"].head()

In [ ]:
bollinger_band_signal_generator(fetch_ohlc(ticker_symbol))

Practice Bollinger

In [ ]:

def bollinger_band(get_dat_data):
    closing_price = pd.DataFrame(get_dat_data(ticker_symbol))
    # Window size
    bollinger_band_window = 20
    num_standard_deviation = 2

    # Calculate mean and standard deviation
    closing_price['bollinger_band_middle'] = closing_price['close'].rolling(window=bollinger_band_window).mean()
    closing_price['bollinger_band_std'] = closing_price['close'].rolling(window=bollinger_band_window).std()

    # Calculate upper bollinger band and lower bollinger band
    closing_price['bollinger_band_upper'] = closing_price['bollinger_band_middle'] + (closing_price['bollinger_band_std'] * num_standard_deviation)
    closing_price['bollinger_band_lower'] = closing_price['bollinger_band_middle'] - (closing_price['bollinger_band_std'] * num_standard_deviation)
    # Create signal for long
    closing_price['bollinger_band_long'] = np.where(closing_price['close'] > closing_price['bollinger_band_upper'], 1.0, 0.0)
    closing_price.loc[closing_price['bollinger_band_long'] == 1.0].head()
    closing_price.dropna(inplace=True)
    # Plot Bollinger Bands with closing prices to visually see data
    return closing_price[['close', 'bollinger_band_middle', 'bollinger_band_upper', 'bollinger_band_lower']].plot(figsize=(20,10), title='AAPL Bollinger Bands')

In [ ]:
bollinger_band(get_dat_data)

ROUGH DRAFT FOR ALPACA API

In [ ]:
# Bringing in alpaca api keys
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version= "v2")

In [ ]:
# testing alpaca keys
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpca Secret Key type: {type(alpaca_secret_key)}")

In [ ]:
# Defining timeframe and ticker parameters for my API pull
current_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

ticker = ["JPM"]

timeframe = "15Min"

In [ ]:
# Creating dataframe with every feature on Alpaca's get_barset
df_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=current_date
).df

df_ticker.head()

In [ ]:
# Creating a clsing price dataframe. Had to get a little weird with this code. 
closing_price = pd.DataFrame()

for time_period in ticker:
    closing_price[x] = df_ticker[time_period]["close"]

closing_price.head()


In [ ]:
import os
import asyncio
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import matplotlib.pyplot as plt


In [ ]:
def execute_trade_strategy(all_signals, account):

    if bollinger_band_signal_generator(fetch_close(ticker_symbol), "close")["long"].iloc == 1.0:
        print(buy)